In [1]:
from pinnsform import *

In [2]:
res1, b_left1, b_right1, b_upper1, b_lower1 = get_data([0,2*np.pi], [0,1], 3, 3)

res2 = make_time_sequence(res1, num_step=4, step=1e-3)
b_left2 = make_time_sequence(b_left1, num_step=4, step=1e-3)
b_right2 = make_time_sequence(b_right1, num_step=4, step=1e-3)
b_upper2 = make_time_sequence(b_upper1, num_step=4, step=1e-3)
b_lower2 = make_time_sequence(b_lower1, num_step=4, step=1e-3)

#res = torch.tensor(res, dtype=torch.float32, requires_grad=True).to(device)
#b_left = torch.tensor(b_left, dtype=torch.float32, requires_grad=True).to(device)
#b_right = torch.tensor(b_right, dtype=torch.float32, requires_grad=True).to(device)
#b_upper = torch.tensor(b_upper, dtype=torch.float32, requires_grad=True).to(device)
#b_lower = torch.tensor(b_lower, dtype=torch.float32, requires_grad=True).to(device)

x_res, t_res = res2[:,:,0:1], res2[:,:,1:2]
x_left, t_left = b_left2[:,:,0:1], b_left2[:,:,1:2]
x_right, t_right = b_right2[:,:,0:1], b_right2[:,:,1:2]
x_upper, t_upper = b_upper2[:,:,0:1], b_upper2[:,:,1:2]
x_lower, t_lower = b_lower2[:,:,0:1], b_lower2[:,:,1:2]

In [16]:
res2

array([[[0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e-03],
        [0.00000000e+00, 2.00000000e-03],
        [0.00000000e+00, 3.00000000e-03]],

       [[3.14159265e+00, 0.00000000e+00],
        [3.14159265e+00, 1.00000000e-03],
        [3.14159265e+00, 2.00000000e-03],
        [3.14159265e+00, 3.00000000e-03]],

       [[6.28318531e+00, 0.00000000e+00],
        [6.28318531e+00, 1.00000000e-03],
        [6.28318531e+00, 2.00000000e-03],
        [6.28318531e+00, 3.00000000e-03]],

       [[0.00000000e+00, 5.00000000e-01],
        [0.00000000e+00, 5.01000000e-01],
        [0.00000000e+00, 5.02000000e-01],
        [0.00000000e+00, 5.03000000e-01]],

       [[3.14159265e+00, 5.00000000e-01],
        [3.14159265e+00, 5.01000000e-01],
        [3.14159265e+00, 5.02000000e-01],
        [3.14159265e+00, 5.03000000e-01]],

       [[6.28318531e+00, 5.00000000e-01],
        [6.28318531e+00, 5.01000000e-01],
        [6.28318531e+00, 5.02000000e-01],
        [6.28318531e+00,

In [10]:
res2.shape

(9, 4, 2)

In [12]:
mesh, boundaries = generate_mesh_object((3,3), domain=[[0,2*np.pi], [0,1]], device="cpu", full_requires_grad=True, border_requires_grad=False)

b_left = boundaries[0][0]
b_right = boundaries[0][1]
initial = boundaries[1][0]

In [13]:
mesh.full

tensor([[0.0000, 0.0000],
        [0.0000, 0.5000],
        [0.0000, 1.0000],
        [3.1416, 0.0000],
        [3.1416, 0.5000],
        [3.1416, 1.0000],
        [6.2832, 0.0000],
        [6.2832, 0.5000],
        [6.2832, 1.0000]], grad_fn=<CatBackward0>)

In [37]:
def meshify_sequence(l, device, requires_grad = False, num_steps = 1, step_size = 0.0):
    temp_np_mesh = np.array(list(product(*l)))
    # if sequence is desired
    if num_steps > 1:
        temp_np_mesh = make_temporal_sequence(temp_np_mesh, num_step=num_steps, step=step_size)
    
    
    print(temp)
    temp_seq = make_temporal_sequence(temp)
    print(temp_seq)
    #seq_parts = [torch.tensor(part, requires_grad = requires_grad, dtype=torch.get_default_dtype()).to(device) for part in np.array_split(temp_seq)]
    parts = [torch.tensor(part, requires_grad = requires_grad, dtype=torch.get_default_dtype()).to(device) for part in np.array_split(np.array(list(product(*l))), 2, 1)]
    full = torch.cat(parts, dim=1)
    return Mesh(full, parts)

def generate_mesh_sequence(point_counts, domain, skew = None, device = 'cpu', full_requires_grad = True, border_requires_grad = False):
    if isinstance(point_counts, int):
        point_counts = [point_counts]
        domain = [domain]

    if skew:
        full_list = [np.linspace(0, 1, count) for i, count in enumerate(point_counts)]
        full_list = [points ** skew[i] for i, points in enumerate(full_list)]
        full_list = [domain[i][0] + (domain[i][1] - domain[i][0]) * points for i,points in enumerate(full_list)]
    else:
        full_list = [np.linspace(domain[i][0], domain[i][1], count) for i, count in enumerate(point_counts)]
    
    #t_list = []
    #for t_value in full_list[1]:
    #    for i in range(5):
    #        t_list.append(t_value+i*1e-4)
    
    #full_list[1] = np.array(t_list)

    print(full_list)

    full_mesh = meshify(full_list, device, full_requires_grad)
    
    #border_lists = [(copy(full_list), copy(full_list)) for _ in full_list]
    #for i, _ in enumerate(full_list):
    #    border_lists[i][0][i] = [float(domain[i][0])]
    #    border_lists[i][1][i] = [float(domain[i][1])]
    #border_meshes = [(meshify(border[0], device, border_requires_grad), meshify(border[1], device, border_requires_grad)) for border in border_lists]
    
    return full_mesh, full_list#, border_meshes

In [38]:
full_list, np_list = generate_mesh_sequence((3,3), domain=[[0,2*np.pi], [0,1]], device="cpu", full_requires_grad=True, border_requires_grad=False)

[array([0.        , 3.14159265, 6.28318531]), array([0. , 0.5, 1. ])]


In [39]:
np_list

[array([0.        , 3.14159265, 6.28318531]), array([0. , 0.5, 1. ])]

In [40]:
array = np.array(list(product(*np_list)))

In [44]:
array.shape

(9, 2)

In [45]:
sequences = make_temporal_sequence(array)

In [46]:
len(sequences.shape)-1

2

In [47]:
sequence_split = np.array_split(sequences, 2, -1)

In [48]:
sequence_split

[array([[[0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265]],
 
        [[3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265]],
 
        [[3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265],
         [3.14159265]],
 
        [[6.28318531],
         [6.28318531],
         [6.28318531],
         [6.28318531],
         [6.28318531]],
 
        [[6.28318531],
         [6.28318531],
         [6.28318531],
         [6.28318531],
         [6.28318531]],
 
        [[6.28318531],
         [6.28318531],
         [

In [49]:
parts = [torch.tensor(part, requires_grad = False, dtype=torch.get_default_dtype()).to('cpu') for part in sequence_split]

In [51]:
parts

[tensor([[[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],
 
         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],
 
         [[0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]],
 
         [[3.1416],
          [3.1416],
          [3.1416],
          [3.1416],
          [3.1416]],
 
         [[3.1416],
          [3.1416],
          [3.1416],
          [3.1416],
          [3.1416]],
 
         [[3.1416],
          [3.1416],
          [3.1416],
          [3.1416],
          [3.1416]],
 
         [[6.2832],
          [6.2832],
          [6.2832],
          [6.2832],
          [6.2832]],
 
         [[6.2832],
          [6.2832],
          [6.2832],
          [6.2832],
          [6.2832]],
 
         [[6.2832],
          [6.2832],
          [6.2832],
          [6.2832],
          [6.2832]]]),
 tensor([[[0.0000e+00],
          [1.0000e-04],
          [2.0000e-04],
 

In [52]:
full = torch.cat(parts, dim=-1)

In [53]:
full

tensor([[[0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04],
         [0.0000e+00, 2.0000e-04],
         [0.0000e+00, 3.0000e-04],
         [0.0000e+00, 4.0000e-04]],

        [[0.0000e+00, 5.0000e-01],
         [0.0000e+00, 5.0010e-01],
         [0.0000e+00, 5.0020e-01],
         [0.0000e+00, 5.0030e-01],
         [0.0000e+00, 5.0040e-01]],

        [[0.0000e+00, 1.0000e+00],
         [0.0000e+00, 1.0001e+00],
         [0.0000e+00, 1.0002e+00],
         [0.0000e+00, 1.0003e+00],
         [0.0000e+00, 1.0004e+00]],

        [[3.1416e+00, 0.0000e+00],
         [3.1416e+00, 1.0000e-04],
         [3.1416e+00, 2.0000e-04],
         [3.1416e+00, 3.0000e-04],
         [3.1416e+00, 4.0000e-04]],

        [[3.1416e+00, 5.0000e-01],
         [3.1416e+00, 5.0010e-01],
         [3.1416e+00, 5.0020e-01],
         [3.1416e+00, 5.0030e-01],
         [3.1416e+00, 5.0040e-01]],

        [[3.1416e+00, 1.0000e+00],
         [3.1416e+00, 1.0001e+00],
         [3.1416e+00, 1.0002e+00],
         [

In [26]:
def make_temporal_sequence(src, num_step=5, step=1e-4):
    dim = num_step
    src = np.repeat(np.expand_dims(src, axis=1), dim, axis=1)  # (N, L, 2)
    for i in range(num_step):
        src[:,i,-1] += step*i
    return src